In [27]:
%pip install selenium parsel

You should consider upgrading via the 'c:\Users\anfiniti\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [28]:
from selenium import webdriver


chromedrive_path = '.\chromedriver.exe' # use the path to the driver you downloaded from previous steps
driver = webdriver.Chrome(chromedrive_path)

AttributeError: 'str' object has no attribute 'capabilities'

In [29]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service  # Import Service untuk mendefinisikan path

# Tentukan path ke Chromedriver
chromedriver_path = './chromedriver.exe'  # Ubah sesuai lokasi file Anda

# Buat objek Service dengan path driver
service = Service(chromedriver_path)

# Inisialisasi webdriver
driver = webdriver.Chrome(service=service)

# Buka halaman web
driver.get("https://www.google.com")
